Taller De Credit Fraud Detection 

In [6]:
import sys
import time

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import numpy as np
import pandas as pd
from IPython.display import HTML

# sys.path.append("code/.")

#import mglearn
from IPython.display import display
#from plotting_functions import *


# Preprocessing and pipeline
from sklearn.impute import SimpleImputer
from scipy.stats import reciprocal

# train test split and cross validation
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    PolynomialFeatures,
)
# pd.set_option("display.max_colwidth", 200)

In [7]:
#Cargamos los datos y procesamos algunas columnas
df = pd.read_csv('creditcard_2023.csv')

#Dado el tamangio del data set se deben eliminar unas columnas y filas repetidas para agilizar el modelamiento 
# Calculate the correlation matrix for numerical columns
correlation_matrix = df.corr()

# Get the correlation of each feature with the "Class" column
correlation_with_class = correlation_matrix["Class"].sort_values(ascending=False)

# Plot a heatmap of the correlation matrix
# plt.figure(figsize=(12, 8))
# sns.heatmap(correlation_matrix, cmap="coolwarm", annot=False)
# plt.title("Correlation Matrix Heatmap")
# plt.show()
# Print the correlations with the "Class" column
#print("Correlation with Class:\n", correlation_with_class)

# Drop any rows that are completely duplicated
df = df.drop_duplicates()

# Print the total number of transactions
print("Total number of transactions:", df.shape[0])
# Print the number of fraudulent transactions
print("Number of fraudulent transactions:", df[df['Class'] == 1].shape[0])
# Print the number of good transactions
print("Number of good transactions:", df[df['Class'] == 0].shape[0])

# Selecting columns where absolute correlation with 'Class' is higher than 0.4
relevant_features = correlation_with_class[correlation_with_class.abs() > 0.4].index
# Dataframe with only relevant features
df_corr = df[relevant_features]

df_corr.to_csv('filtered_creditcard_2023.csv', index=False)
# print(df.head())

columns_to_consider = [col for col in df_corr.columns if col not in ['id', 'Amount', 'Class']]

# Drop duplicates, considering only the columns in 'columns_to_consider'
df_corr.drop_duplicates(subset=columns_to_consider, inplace=True)

# Print the total number of transactions
print("Total number of transactions after drop:", df_corr.shape[0])
# Print the number of fraudulent transactions
print("Number of fraudulent transactions after drop:", df_corr[df_corr['Class'] == 1].shape[0])
# Print the number of good transactions
print("Number of good transactions after drop:", df_corr[df_corr['Class'] == 0].shape[0])

Total number of transactions: 568630
Number of fraudulent transactions: 284315
Number of good transactions: 284315
Total number of transactions after drop: 552035
Number of fraudulent transactions after drop: 276845
Number of good transactions after drop: 275190


C:\Users\sebas\AppData\Local\Temp\ipykernel_31620\2783982765.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr.drop_duplicates(subset=columns_to_consider, inplace=True)


In [8]:
#Separar los datos en entrenamiento, validación y test
train_df, test_df = train_test_split(df_corr, test_size=0.2, random_state=1)

#Definir características y variable objetivo
X_train = train_df.drop(columns=["Class"])
y_train = train_df["Class"]

X_test = test_df.drop(columns=["Class"])
y_test = test_df["Class"]

# #Índices de las columnas numéricas y categ+oricas
# cat_cols = X_train.select_dtypes(include=object).columns
# num_cols = X_train.select_dtypes(include=np.number).columns

In [9]:
#Se seleccionan todas las columnas 
# num_cols = X_train.select_dtypes(include=np.number).columns

# #Se hace un standard Scaler aunque los datos ya tienen un escalamiento.
# numeric_transformer = Pipeline(
#     steps=[("scaler", StandardScaler())]
# )

# #Se crea el preprocesador 
# preprocessor = ColumnTransformer(
#     transformers=[
#         ("num", numeric_transformer, num_cols)
#     ]
# )

In [10]:
#Definimos el clasificador base
logreg_base = LogisticRegression()
clf_logreg = Pipeline(steps=[("classifier", logreg_base)])
# clf_logreg = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", logreg_base)])

param_grid = {
    'classifier__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'classifier__C': np.logspace(-4, 4, 20),
    'classifier__penalty': ['l1', 'l2', 'elasticnet', 'none'],
}

search_logreg = GridSearchCV(clf_logreg, param_grid, scoring='accuracy', n_jobs=-1)

search_logreg.fit(X_train, y_train)
print(search_logreg)
print(search_logreg.best_params_)

# Obtengamos el accuracy para la regresión logística 
print('Modelo regresión logística')
print(f'Accuracy: {search_logreg.score(X_test, y_test)}')

KeyboardInterrupt: 